# Aula 15 - RNNs, LSTM e GRU (Continuação) - GRU

## Estrutura das GRUs

### Introdução às GRUs

#### Conceito de Gated Recurrent Units (GRU) - Unidades recorrentes fechadas
Gated Recurrent Units (GRUs) são um tipo de rede neural recorrente (RNN) projetada para resolver problemas comuns encontrados em RNNs tradicionais, como o desvanecimento e a explosão de gradientes. As GRUs utilizam mecanismos de gating (portões) para controlar o fluxo de informações através das células recorrentes, permitindo que a rede armazene e transporte informações ao longo de sequências temporais de forma mais eficiente.

#### Histórico e desenvolvimento das GRUs
As GRUs foram introduzidas por Kyunghyun Cho et al. em 2014 como uma alternativa simplificada às redes Long Short-Term Memory (LSTM). O principal objetivo era criar uma estrutura de rede recorrente que fosse menos complexa e mais rápida de treinar, mantendo, ao mesmo tempo, a capacidade de capturar dependências temporais de longo prazo. Desde sua introdução, as GRUs têm sido amplamente adotadas em várias aplicações de processamento de linguagem natural (PLN), reconhecimento de fala e previsão de séries temporais devido à sua eficiência e desempenho.

### Componentes das GRUs

#### Gates nas GRUs
As GRUs utilizam dois tipos principais de gates para controlar o fluxo de informações: o Update Gate e o Reset Gate. Esses gates são essenciais para o funcionamento eficiente das GRUs, permitindo que a rede aprenda dependências temporais de longo prazo de maneira mais eficaz.

##### Update Gate
- **Função:** O Update Gate controla a quantidade de informação do estado anterior que deve ser transportada para o próximo estado.
- **Importância:** Este gate determina quais informações são importantes o suficiente para serem mantidas ao longo das etapas de tempo, ajudando a mitigar o problema do desvanecimento de gradientes.

##### Reset Gate
- **Função:** O Reset Gate controla a quantidade de informação do estado anterior que deve ser esquecida.
- **Importância:** Este gate permite que a rede decida quais informações antigas não são mais relevantes e devem ser descartadas, focando assim nas novas entradas.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Definição do modelo GRU
model = Sequential([
    # Primeira camada GRU
    # units: número de unidades (neurônios) na camada GRU
    # return_sequences: se True, retorna a sequência completa de saídas
    # input_shape: formato da entrada (timesteps, features)
    # activation: função de ativação para as saídas
    GRU(units=50, return_sequences=True, input_shape=(None, 1), activation='tanh'),

    # Segunda camada GRU
    # units: número de unidades (neurônios) na camada GRU
    # return_sequences: se False, retorna apenas a última saída
    GRU(units=50, return_sequences=False, activation='tanh'),

    # Camada densa
    # units: número de unidades na camada (aqui é 1, indicando saída unidimensional)
    Dense(units=1, activation='linear')
])

# Compilação do modelo
# optimizer: algoritmo de otimização (Adam é um dos mais usados)
# loss: função de perda (MSE é comum para problemas de regressão)
model.compile(optimizer='adam', loss='mse')

# Resumo do modelo
model.summary()


### Funcionamento das GRUs

#### Cálculo do Update Gate (z)
- **Fórmula Matemática:** 
 
  $( z_t = \sigma(W_z \cdot [h_{t-1}, x_t]) )$
- **Explicação dos Pesos e Ativações:**
  - $( W_z )$ representa os pesos associados ao Update Gate.
  - $( h_{t-1} )$ é o estado oculto anterior.
  - $( x_t )$ é a entrada atual.
  - $( \sigma )$ é a função sigmoide que restringe os valores entre 0 e 1.
  - O Update Gate decide a quantidade de informação anterior (\( h_{t-1} \)) a ser transportada para o próximo estado.

#### Cálculo do Reset Gate (r)
- **Fórmula Matemática:** 
  $( r_t = \sigma(W_r \cdot [h_{t-1}, x_t]) )$
- **Explicação dos Pesos e Ativações:**
  - $( W_r )$ representa os pesos associados ao Reset Gate.
  - $( h_{t-1} )$ é o estado oculto anterior.
  - $( x_t )$ é a entrada atual.
  - $( \sigma )$ é a função sigmoide que restringe os valores entre 0 e 1.
  - O Reset Gate decide a quantidade de informação anterior ($( h_{t-1} )$) a ser esquecida.

#### Cálculo do Novo Estado do Candidato ($( \tilde{h} )$)
- **Fórmula Matemática:** 
  $( \tilde{h}_t = \tanh(W \cdot [r_t \odot h_{t-1}, x_t]) )$
- **Explicação dos Pesos e Ativações:**
  - $( W )$ representa os pesos associados ao novo estado do candidato.
  - $( r_t )$ é o valor do Reset Gate.
  - $( h_{t-1} )$ é o estado oculto anterior.
  - $( x_t )$ é a entrada atual.
  - $( \odot )$ representa a multiplicação elemento a elemento (Hadamard product).
  - $( \tanh )$ é a função tangente hiperbólica que restringe os valores entre -1 e 1.
  - O novo estado do candidato ($( \tilde{h}_t )$) é calculado considerando a informação relevante filtrada pelo Reset Gate.

#### Cálculo do Novo Estado Oculto (h)
- **Fórmula Matemática:** 
  $( h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t )$
- **Explicação da Combinação Linear de Estados:**
  - $( z_t )$ é o valor do Update Gate.
  - $( h_{t-1} )$ é o estado oculto anterior.
  - $( \tilde{h}_t )$ é o novo estado do candidato.
  - $( \odot )$ representa a multiplicação elemento a elemento (Hadamard product).
  - O novo estado oculto ($( h_t )$) é uma combinação linear do estado anterior e do novo estado do candidato, ponderada pelos valores do Update Gate. Esta combinação permite que a rede decida quanta informação do passado deve ser transportada para o futuro.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Microsoft_Stock.csv')

In [ ]:
df.info()

In [ ]:
# Converte a coluna Date para o tipo de dados de data
df['Date'] = pd.to_datetime(df['Date'])

# Ordena os dados em ordem crescente pelos dado
df = df.sort_values(by='Date')

# Exclui a coluna Date e deixa a classe alvo como última coluna
df = df[['Open', 'High', 'Low', 'Volume', 'Close']]


In [ ]:
df.info()

In [ ]:
import numpy as np

def create_lookback_data(df, lookback):
    """
    Função para criar dados de entrada e saída para uma LSTM usando uma janela de lookback.
    
    Parâmetros:
    df (DataFrame): DataFrame contendo os dados. A última coluna é a coluna alvo.
    lookback (int): Número de dias para a janela de lookback.
    
    Retorna:
    X (numpy array): Dados de entrada para a LSTM, de formato (n_amostras, lookback, n_features).
    y (numpy array): Dados de saída/targets, de formato (n_amostras,).
    """
    data = df.values
    X, y = [], []
    
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i, :])  # Todas as colunas exceto a última (características)
        y.append(data[i, -1])               # Apenas a última coluna (alvo)
    
    return np.array(X), np.array(y)

In [ ]:
lookback = 20
X, y = create_lookback_data(df, lookback)

print(f"Formato de X: {X.shape}")
print(f"Formato de y: {y.shape}")

Os dados de X.shape informam que X tem 1491 registros, dos quais cada um possui 20 linhas e 5 colunas (3 dimensões)

In [ ]:
# Separar os dados em treinamento e teste
X_train = X[:int(len(X) * 0.8)]
X_test = X[int(len(X) * 0.8):]

y_train = y[:int(len(y) * 0.8)]
y_test = y[int(len(y) * 0.8):]

In [ ]:
X_train.shape

In [ ]:
# Informa que tem 23840 registros com cada registro 5 colunas (valores) - 2 dimensões
X_train.reshape(-1, X_train.shape[-1]).shape

In [ ]:
1192 * 20

In [ ]:
X_train.reshape(-1, X_train.shape[-1])[0]

In [ ]:
X_train[0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalizando os dados de treinamento
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

In [ ]:
X_train = scaler_X.fit_transform(X_train)

In [ ]:
# Ajustando a forma de X_train para duas dimensões antes da normalização
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
print(X_train_reshaped.shape)
X_train_reshaped[0]

In [ ]:
# Normaliza os valores
X_train_scaled = scaler_X.fit_transform(X_train_reshaped)
print(X_train_scaled.shape)
X_train_scaled[0]

In [ ]:
# Ajustando a forma de volta para três dimensões após a normalização
X_train = X_train_scaled.reshape(X_train.shape)
print(X_train.shape)
X_train[0]

In [ ]:
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))

# Normalizando os dados de teste
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
y_test = scaler_y.transform(y_test.reshape(-1, 1))

## Comparação GRU vs. LSTM

### Estrutura das LSTM

#### Componentes das LSTM
As LSTM são redes neurais recorrentes projetadas para modelar dependências de longo prazo, superando limitações de RNNs tradicionais.

##### Input Gate
- **Função:** Controla a quantidade de nova informação da entrada atual ($x_t$) a ser armazenada no estado da célula.
- **Composição:**
  - Pesos associados à entrada atual ($x_t$) e ao estado oculto anterior ($h_{t-1}$).
  - Função de ativação sigmoide ($\sigma$) que gera valores entre 0 e 1.
- **Fórmula Matemática:**
  - $i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$

##### Forget Gate
- **Função:** Controla a quantidade de informação anterior a ser esquecida do estado da célula.
- **Composição:**
  - Pesos associados à entrada atual ($x_t$) e ao estado oculto anterior ($h_{t-1}$).
  - Função de ativação sigmoide ($\sigma$) que gera valores entre 0 e 1.
- **Fórmula Matemática:**
  - $f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$

##### Output Gate
- **Função:** Controla a quantidade de informação do estado da célula a ser transferida para o estado oculto atual ($h_t$).
- **Composição:**
  - Pesos associados à entrada atual ($x_t$) e ao estado oculto anterior ($h_{t-1}$).
  - Função de ativação sigmoide ($\sigma$) que gera valores entre 0 e 1.
  - Função tangente hiperbólica ($\tanh$) que regula a amplitude da saída.
- **Fórmula Matemática:**
  - $o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$
  - $h_t = o_t \odot \tanh(C_t)$

##### Cell State
- **Função:** Armazena informações a longo prazo, modificadas pelas gates de entrada e de esquecimento.
- **Composição:**
  - Combinação linear do estado anterior da célula e da nova informação ajustada pelas gates.
- **Fórmula Matemática:**
  - $C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$
  - $\tilde{C}_t$ é o novo estado candidato, calculado como $ \tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$

### Diferenças chave entre GRUs e LSTMs

#### Número de Gates
- **LSTMs:**
  - **Input Gate:** Controla a quantidade de nova informação a ser armazenada no estado da célula.
  - **Forget Gate:** Controla a quantidade de informação anterior a ser esquecida do estado da célula.
  - **Output Gate:** Controla a quantidade de informação do estado da célula a ser transferida para o estado oculto atual.
- **GRUs:**
  - **Update Gate:** Controla a quantidade de informação anterior a ser transportada para a próxima etapa.
  - **Reset Gate:** Controla a quantidade de informação anterior a ser esquecida.

As LSTMs possuem três gates principais (input, forget, output), enquanto as GRUs possuem dois gates principais (update, reset). Esta diferença resulta em uma arquitetura mais simples para as GRUs.

#### Complexidade Computacional
- **LSTMs:**
  - Devido à presença de três gates, as LSTMs são mais complexas e exigem mais cálculos em cada etapa de tempo.
  - A complexidade adicional das LSTMs pode resultar em tempos de treinamento mais longos e maior uso de recursos computacionais.
- **GRUs:**
  - Com apenas dois gates, as GRUs são menos complexas e realizam menos cálculos em cada etapa de tempo.
  - A arquitetura mais simples das GRUs permite treinos mais rápidos e uso eficiente dos recursos computacionais.

As GRUs tendem a ser mais rápidas e menos complexas devido ao menor número de gates, tornando-as uma escolha eficiente para muitas aplicações práticas.

#### Eficiência de Treinamento
- **LSTMs:**
  - Requerem uma quantidade maior de dados de treinamento para aprender eficazmente, devido à sua complexidade.
  - A presença de múltiplos gates pode tornar o ajuste de hiperparâmetros mais desafiador.
- **GRUs:**
  - Geralmente requerem menos dados de treinamento para alcançar um desempenho comparável, devido à sua estrutura mais simples.
  - A menor complexidade das GRUs facilita o processo de treinamento e ajuste de hiperparâmetros.

GRUs geralmente requerem menos dados para alcançar desempenho comparável, facilitando o treinamento em cenários com dados limitados.

#### Efetividade
- **Desempenho em Tarefas Específicas:**
  - **LSTMs:** Excelentes em tarefas que exigem a captura de dependências de longo prazo, como tradução automática e geração de texto.
  - **GRUs:** Desempenham de forma similar às LSTMs em muitas tarefas, mas com vantagens em termos de velocidade e eficiência em cenários onde os dados são menos complexos ou de menor volume.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
%%time
# Definindo o modelo RNN
model_rnn = Sequential([
    SimpleRNN(50, return_sequences=True, input_shape=(lookback, X_train.shape[-1]), activation='tanh'),
    Dropout(0.2),
    SimpleRNN(50, return_sequences=False, activation='tanh'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

model_rnn.compile(optimizer='adam', loss='mse')
history_rnn = model_rnn.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=32, verbose=1)

In [ ]:
%%time
# Definindo o modelo LSTM
model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(lookback, X_train.shape[-1]), activation='tanh'),
    Dropout(0.2),
    LSTM(50, return_sequences=False, activation='tanh'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

model_lstm.compile(optimizer='adam', loss='mse')
history_lstm = model_lstm.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=32, verbose=1)

In [ ]:
%%time
# Definindo o modelo GRU
model_gru = Sequential([
    GRU(50, return_sequences=True, input_shape=(lookback, X_train.shape[-1]), activation='tanh'),
    Dropout(0.2),
    GRU(50, return_sequences=False, activation='tanh'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

model_gru.compile(optimizer='adam', loss='mse')
history_gru = model_gru.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=32, verbose=1)

In [ ]:
plt.figure(figsize=(14, 8))

# Perdas do modelo RNN
plt.plot(history_rnn.history['loss'], label='Train Loss RNN')
plt.plot(history_rnn.history['val_loss'], label='Val Loss RNN')

# Perdas do modelo LSTM
plt.plot(history_lstm.history['loss'], label='Train Loss LSTM')
plt.plot(history_lstm.history['val_loss'], label='Val Loss LSTM')

# Perdas do modelo GRU
plt.plot(history_gru.history['loss'], label='Train Loss GRU')
plt.plot(history_gru.history['val_loss'], label='Val Loss GRU')

plt.title('Training and Validation Loss for RNN, LSTM, and GRU')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Função para calcular e imprimir métricas
def evaluate_model(model, X_test, y_test, scaler_y):
    y_pred = model.predict(X_test)
    y_pred_inverse = scaler_y.inverse_transform(y_pred)
    y_test_inverse = scaler_y.inverse_transform(y_test)
    mse = mean_squared_error(y_test_inverse, y_pred_inverse)
    mae = mean_absolute_error(y_test_inverse, y_pred_inverse)
    r2 = r2_score(y_test_inverse, y_pred_inverse)
    return mse, mae, r2, y_test_inverse, y_pred_inverse

In [ ]:
# Avaliando o modelo RNN
mse_rnn, mae_rnn, r2_rnn, y_test_rnn, y_pred_rnn = evaluate_model(model_rnn, X_test, y_test, scaler_y)

# Avaliando o modelo LSTM
mse_lstm, mae_lstm, r2_lstm, y_test_lstm, y_pred_lstm = evaluate_model(model_lstm, X_test, y_test, scaler_y)

# Avaliando o modelo GRU
mse_gru, mae_gru, r2_gru, y_test_gru, y_pred_gru = evaluate_model(model_gru, X_test, y_test, scaler_y)

In [ ]:
print(f'RNN - MSE: {mse_rnn}, MAE: {mae_rnn}, R2: {r2_rnn}')
print(f'LSTM - MSE: {mse_lstm}, MAE: {mae_lstm}, R2: {r2_lstm}')
print(f'GRU - MSE: {mse_gru}, MAE: {mae_gru}, R2: {r2_gru}')

### Interpretação do Coeficiente de Determinação (R²)

- **R² = 1**: O modelo explica perfeitamente a variação nos dados.
- **R² entre 0 e 1**: O modelo explica parcialmente a variação nos dados.
- **R² = 0**: O modelo não explica nenhuma variação nos dados.
- **R² negativo**: O modelo está pior do que uma linha horizontal (o modelo de média), ou seja, a previsão média teria sido uma melhor estimativa do que o modelo preditivo.


# Exercícios em um outro arquivo ;)